In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
import numpy as np
import imageio
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import keras 
import cv2
from sklearn.utils import shuffle

In [0]:
import random
random.seed(2) # Python
np.random.seed(1) #numpy
from tensorflow import set_random_seed
set_random_seed(3) # Tensorflow

In [0]:
#The path to the omniglot data
root_path = "./drive/My Drive/"
train_path = os.path.join(root_path,'Traning') 
validation_path = os.path.join(root_path,'Evalution')

def load_image(path, n = 0):
    X = []
    
    #Load every alphabet seperately and place that in one tensor
    for alphabet in os.listdir(path):
#         print("Loading Alphabet: " + alphabet)
        alphabet_path = os.path.join(path, alphabet)
       
        category_images = []
            
        if not os.path.isdir(alphabet_path):
                continue
            
            #Read evey image with in the directory
        for filename in os.listdir(alphabet_path):
                image_path = os.path.join(alphabet_path, filename)
                image = imageio.imread(image_path)
                width = 105
                height = 105 # keep original height
                dim = (height,width)
 
                # resize image
                image = cv2.resize(image, dim)
                image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image= np.expand_dims(image, axis=0)
#                 print(image_path)
                #Image preprocessing
                image = image/255
                image = 1 - image
                
                X.append(image)
                
        
    X = np.stack(X)
    return X

In [193]:
print("Loading Training Set")
Xtrain = load_image(train_path)
print(Xtrain.shape)

Loading Training Set
(1064, 1, 105, 105)


In [194]:
print("Now loading evaluation set")
Xval = load_image(validation_path)
print(Xval.shape)

Now loading evaluation set
(1064, 1, 105, 105)


In [0]:
def get_batch(data, batch_size):
    n_classes, n_examples, h, w = data.shape
    
    pairs = [np.zeros((batch_size, 1, h, w)) for i in range(2)]
    
    targets = np.zeros((batch_size,))
    targets[batch_size//2:] = 1
    
    categories = np.random.choice(n_classes, size = (batch_size), replace = False)
    
    for i in range(batch_size):
        category = categories[i]
        
        idx1 = np.random.randint(0, n_examples)
        pairs[0][i,:,:,:] = data[category, idx1].reshape(1, h,w)
        idx2 = np.random.randint(0, n_examples)
        
        if targets[i] == 0:
            category_2 = category
        else:
            category_2 = (category + np.random.randint(1, n_classes)) % n_classes
 
        
        pairs[1][i,:,:,:] = data[category_2, idx2].reshape(1, h, w)
        
    return pairs, targets
        



In [0]:
def generate(data,batch_size):
        """a generator for batches, so model.fit_generator can be used. """
        while True:
            pairs, targets = get_batch(Xtrain,batch_size)
            yield (pairs, targets)    


 

In [0]:
def make_oneshot_task(N,data,language=None):
        """Create pairs of test image, support set for testing N way one-shot learning. """
        n_classes, n_examples, w, h = data.shape
        indices = np.random.randint(0,n_examples,size=(N,))
        categories = np.random.choice(range(n_classes),size=(N,),replace=False)            
        true_category = categories[0]
        ex1 = np.random.randint(0, n_examples)
        ex2 = np.random.randint(0, n_examples)
        # ex1, ex2 = np.random.choice(n_examples,replace=False,size=(0,))
        test_image = np.asarray([data[true_category,ex1,:,:]]*N).reshape(N, 1, w,h)
        support_set = data[categories,indices,:,:]
        support_set[0,:,:] = data[true_category,ex2]
        support_set = support_set.reshape(N, 1, w,h)
        targets = np.zeros((N,))
        targets[0] = 1
        targets, test_image, support_set = shuffle(targets, test_image, support_set)
        pairs = [test_image,support_set]

        return pairs, targets
    

                            

In [0]:
def test_oneshot(model,N,k,data,verbose=0):
        """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
        n_correct = 0
        if verbose:
            print("Evaluating model on {} random {} way one-shot learning tasks ...".format(k,N))
        for i in range(k):
            inputs, targets = make_oneshot_task(N,data)
            probs = model.predict(inputs)
            if np.argmax(probs) == np.argmax(targets):
                n_correct+=1
        percent_correct = (100.0*n_correct / k)
        if verbose:
            print("Got an average of {}% {} way one-shot learning accuracy".format(percent_correct,N))
        return percent_correct
    


In [0]:
def train(model, epochs, verbosity):
        model.fit_generator(self.generate(batch_size),)

In [0]:
from keras.models import Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Input, Lambda
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

In [0]:
#Contrastive Loss
def euclid_dist(input_pair):
    x, y = input_pair
    distance = K.sqrt(K.maximum(K.sum(K.square(x - y), axis=-1, keepdims=True), K.epsilon()))
    return distance

def euclid_dist_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    y_true = -1 * y_true + 1
    return K.mean((1-y_true) * K.square(y_pred) + y_true *  K.square(K.maximum(margin - y_pred, 0.0)))

def acc(y_true, y_pred):
    ones = K.ones_like(y_pred)
    return K.mean(K.equal(y_true, ones - K.clip(K.round(y_pred), 0, 1)), axis=1)



In [0]:
im_width  =105
im_height =105
im_chan   =1      # Number of channels: first is original and second cumsum(axis=0)

In [206]:
inp =Input(( im_chan,im_height,im_width ), name='img')

# print(input_img)
# Down Block 1
c1 = Conv2D(32,(1,1), activation='relu', kernel_regularizer=l2(2e-4) )(input_img)
p1 = MaxPooling2D(data_format="channels_first") (c1)

#Down Block 2
c2 = Conv2D(128, (1, 1), activation='relu', kernel_regularizer=l2(2e-4)) (p1)
p2 = MaxPooling2D(data_format="channels_first") (c2)

#Down Block 3
c3 = Conv2D(128, (1, 1), activation='relu', kernel_regularizer=l2(2e-4)) (p2)
p3 = MaxPooling2D(data_format="channels_first") (c3)

#Down Block 4
c4 = Conv2D(256, (1, 1), activation='relu', kernel_regularizer=l2(2e-4)) (p3)
p4 = MaxPooling2D(data_format="channels_first") (c4)
# # print(p4)
flat = Flatten()(p4)
output = Dense(100, activation='sigmoid')(flat)

# Instantiate the Model
model = Model(input_img, output)

input1 = Input(shape = (1,105,105))
input2 = Input(shape = (1,105,105))

output1 = model(input1)
output2 = model(input2)


distance = Lambda(euclid_dist, output_shape=euclid_dist_shape)([output1, output2])

prediction = Dense(1, activation='sigmoid')(distance)
    
siamese = Model(inputs=[input1, input2], outputs=prediction)
    
optimizer = 'adam'
    
siamese.compile(loss=contrastive_loss, optimizer=optimizer, metrics=[acc])

siamese.summary()

Model: "model_41"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, 1, 105, 105)  0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, 1, 105, 105)  0                                            
__________________________________________________________________________________________________
model_40 (Model)                (None, 100)          107428      input_31[0][0]                   
                                                                 input_32[0][0]                   
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, 1)            0           model_40[1][0]            

In [211]:
#Training loop
print("!")
evaluate_every = 1 # interval for evaluating on one-shot tasks
loss_every=50 # interval for printing loss (iterations)
batch_size = 32
n_iter = 30
N_way = 50 # how many classes for testing one-shot tasks>
n_val = 30 #how mahy one-shot tasks to validate on?
best = -1
weights_path = "weights"
print("training")
for i in range(1, n_iter):
    (inputs,targets)=get_batch(Xtrain,batch_size)
    loss=siamese.train_on_batch(inputs,targets)
    print(loss)
    if i % evaluate_every == 0:
        print("evaluating")
        val_acc = test_oneshot(siamese,N_way,n_val,Xval,verbose=True)
        if val_acc >= best:
            print("saving")
            siamese.save(weights_path)
            best=val_acc

    if i % loss_every == 0:
        print("iteration {}, training loss: {:.2f},".format(i,loss))

!
training
[0.20824057, 1.0]
evaluating
Evaluating model on 30 random 50 way one-shot learning tasks ...
Got an average of 100.0% 50 way one-shot learning accuracy
saving
[0.21306208, 1.0]
evaluating
Evaluating model on 30 random 50 way one-shot learning tasks ...
Got an average of 96.66666666666667% 50 way one-shot learning accuracy
[0.20071301, 1.0]
evaluating
Evaluating model on 30 random 50 way one-shot learning tasks ...
Got an average of 96.66666666666667% 50 way one-shot learning accuracy
[0.19737765, 1.0]
evaluating
Evaluating model on 30 random 50 way one-shot learning tasks ...
Got an average of 96.66666666666667% 50 way one-shot learning accuracy
[0.20325865, 1.0]
evaluating
Evaluating model on 30 random 50 way one-shot learning tasks ...
Got an average of 90.0% 50 way one-shot learning accuracy
[0.18946348, 1.0]
evaluating
Evaluating model on 30 random 50 way one-shot learning tasks ...
Got an average of 96.66666666666667% 50 way one-shot learning accuracy
[0.19587812, 1.0]